In [1]:
import warnings
import itertools
warnings.filterwarnings("ignore")
import pandas as pd
from simple_salesforce import Salesforce
from simple_salesforce import SalesforceMalformedRequest
from tqdm import tqdm
from collections import defaultdict
import glob
import os
from tqdm import tqdm

proxies = {
}

username = ''
password = ''
isSandbox = 0
token = ''
instance = 'https://login.salesforce.com'

sf = Salesforce(username = username,
                password = password,
                security_token = token,
                sandbox = isSandbox,
                proxies = proxies)

Basic data manipulation from database to get to problem at hand

In [64]:
all_districts = sf.query_all("SELECT District__c,Id,Name FROM Territory__c WHERE Object__c = 'Account Location;Lead'")
all_districts = pd.DataFrame.from_records(all_districts['records'])
# query all districts and towns from database

all_districts['District__c'] = all_districts.District__c.map(lambda x : x.split('-')[0])
# get specific district code from string

all_districts['Territory_Code'] = all_districts.Name.map(lambda x : x.split('-')[1])
all_districts['Name[0]'] = all_districts.Name.map(lambda x : x.split('-')[0])
# the 'Name' field is actually a combination of a town code and territory code (splitting them up)

bad_districts = all_districts.groupby('District__c').filter(lambda x : all(x['District__c'] == x['Name[0]'])==False)
# this project has been done manually on some territories, filter these out

bad_districts = bad_districts[['District__c','Name','Name[0]','Territory_Code']]
bad_districts = bad_districts[bad_districts['Territory_Code'].isin(['Vacant','CCNO'])==False]

Problem: 1) The first part of every Name should be the district it is under, but this cannot result in any duplicates. 
         2) Every district should start with a territory code of 1, and increase sequentially (many do not actually start at 1, and many skip random numbers throughout)
         3) Make as few changes to the territory code as possible, however, maintaining original order trumps this
         
         
         i.e. in the below example for district D322
             Name '0096-CC02' would be changed to 'D322-CC08'
             if you did this for every name, because there are 3 territories in the district it would result in mult. duplicates
             

In [65]:
# example
bad_districts[bad_districts['District__c'] == 'D322'].sort_values('Name')

,District__c,Name,Name[0],Territory_Code
57,D322,0096-CC02,0096,CC02
473,D322,0096-CC04,0096,CC04
474,D322,0096-CC06,0096,CC06
210,D322,0136-CC01,0136,CC01
59,D322,0136-CC06,0136,CC06
211,D322,0136-CC08,0136,CC08
372,D322,0480-CC01,0480,CC01
49,D322,0480-CC02,0480,CC02
373,D322,0480-CC03,0480,CC03
517,D322,0480-CC06,0480,CC06


In [66]:
multTowns = bad_districts.groupby('District__c').filter(lambda x : x['Name[0]'].nunique()>1)

In [67]:
# to better visualize this and start the process..
# groupy district and territory -- for every territory remove the 'CC' and turn into an integer, this is all that really matters
# turn this into a list and then a set for better visualization

multTowns.groupby(['District__c','Name[0]']).apply(lambda x : set([x[1] for x in list(x['Territory_Code'].str.split('CC'))])).head(17)

District__c  Name[0]
D140         0336                                       {05}
             0745                               {05, 06, 07}
             D140                           {04, 03, 01, 02}
D148         0122                                   {06, 01}
             0149                       {03, 04, 05, 06, 02}
             0315                           {09, 08, 10, 07}
D233         0081               {03, 04, 07, 05, 06, 01, 02}
             0346                                   {09, 03}
D234         0114                               {03, 01, 02}
             0240                               {04, 05, 03}
D240         0005       {08, 03, 04, 05, 01, 06, 09, 07, 02}
             0701                   {03, 04, 05, 06, 01, 02}
D256         0303                                       {04}
             0942                                   {01, 02}
D322         0096                               {04, 06, 02}
             0136                               {08, 06, 01}
   

You can see above that for district D240, you would want to keep territory codes for 0005 the same, and change 0701 to increment up (this will result in the least change)
For district D140 you would want 0745 to remain and only change 0036

Those two examples result in the following process:

a) Groupyby district and town, turn each town code into integers in a sorted list
b) Sort those lists on two levels
    1) by the first (smallest) number in the list
    2) by the len of the list its self (reversed) // longer lists go first
    3) iterate over the lists making the codes sequential

In [28]:
# example..

In [68]:
test['D322']

Name[0]
0096        [02, 04, 06]
0136        [01, 06, 08]
0480    [01, 02, 03, 06]
dtype: object

In [69]:
# combine the lists for every territory in a district
list(zip(test['D322'].index.tolist(), test['D322'].tolist()))

[('0096', ['02', '04', '06']),
 ('0136', ['01', '06', '08']),
 ('0480', ['01', '02', '03', '06'])]

In [70]:
# sort the lists by the smallest number first, and then by the length of the list itself (reversed)
sorted(list(zip(test['D322'].index.tolist(), test['D322'].tolist())), key= lambda x : (x[1][0],-len(x[1])))

[('0480', ['01', '02', '03', '06']),
 ('0136', ['01', '06', '08']),
 ('0096', ['02', '04', '06'])]

In [71]:
# simple function to return ints back to territory codes 'CC12' etc.

def territory_code(num):
    if num < 10:
        return 'CC0' + str(num)
    else:
        return 'CC' + str(num)

In [72]:
terr_dict = dict()
counter = 0

grouped_districts = bad_districts.groupby(['District__c','Name[0]']).apply(lambda x : sorted([try_int(x[1]) for x in list(x['Territory_Code'].str.split('CC'))]))
# group district ant town, split on 'CC' and turn remainer into and into an integer in a list

for district in grouped_districts.index.levels[0]:
    
    grouped_series = grouped_districts[district]
    # iterate over every district, which is a pandas series
        
    town_lists = list(zip(grouped_series.index.tolist(), grouped_series.tolist()))
    # combine the lists for every territory in a district
    
    sorted_town_list = sorted(town_lists, key= lambda x : (x[1][0],-len(x[1])))
    # sort the lists by the smallest number first, and then by the length of the list itself (reversed)
    
    loop = 0
    
    check = []

    for town in sorted_town_list:
        for code in town[1]:
            loop += 1
            # iterate over every code in the combined sorted town lists
            if code in check:
                # if the code already exists, then you are in a new town
                # and to seqentially increase add one to the end of the current list
                c = check[-1] + 1
                check.append(c)
                terr_dict[town[0] + '-' + territory_code(code)] = town[0] + '-' + territory_code(c)
                # create a dictionary of the previous code with new code to map to original data
                # this will maintain important relationships
            elif loop != code:
                # if the iteration count does not equal the code, then the territory did not start at 1
                c = loop
                check.append(c)
                terr_dict[town[0] + '-' + territory_code(code)] = town[0] + '-' + territory_code(c)
            else:
                check.append(code)

In [73]:
def try_code(row):
    try:
        return row.District__c + '-' + terr_dict[row.Name].split('-')[1]
    except:
        return row.District__c + '-' + row.Territory_Code

bad_districts['new_code'] =bad_districts.apply(try_code, axis=1)

In [74]:
bad_districts[bad_districts['District__c'] =='D322'].sort_values('new_code')

,District__c,Name,Name[0],Territory_Code,new_code
372,D322,0480-CC01,0480,CC01,D322-CC01
49,D322,0480-CC02,0480,CC02,D322-CC02
373,D322,0480-CC03,0480,CC03,D322-CC03
517,D322,0480-CC06,0480,CC06,D322-CC04
210,D322,0136-CC01,0136,CC01,D322-CC05
59,D322,0136-CC06,0136,CC06,D322-CC06
211,D322,0136-CC08,0136,CC08,D322-CC07
57,D322,0096-CC02,0096,CC02,D322-CC08
473,D322,0096-CC04,0096,CC04,D322-CC09
474,D322,0096-CC06,0096,CC06,D322-CC10


In [75]:
bad_districts[bad_districts['District__c'] =='D673'].sort_values('new_code')

,District__c,Name,Name[0],Territory_Code,new_code
471,D673,IRVI-CC01,IRVI,CC01,D673-CC01
824,D673,IRVI-CC02,IRVI,CC02,D673-CC02
825,D673,IRVI-CC03,IRVI,CC03,D673-CC03
826,D673,IRVI-CC04,IRVI,CC04,D673-CC04
827,D673,IRVI-CC05,IRVI,CC05,D673-CC05
828,D673,IRVI-CC06,IRVI,CC06,D673-CC06
829,D673,IRVI-CC07,IRVI,CC07,D673-CC07
831,D673,IRVI-CC09,IRVI,CC09,D673-CC08
832,D673,IRVI-CC10,IRVI,CC10,D673-CC09
833,D673,IRVI-CC11,IRVI,CC11,D673-CC10


In [76]:
bad_districts[bad_districts['District__c'] =='D233'].sort_values('new_code')

,District__c,Name,Name[0],Territory_Code,new_code
360,D233,0081-CC01,0081,CC01,D233-CC01
361,D233,0081-CC02,0081,CC02,D233-CC02
362,D233,0081-CC03,0081,CC03,D233-CC03
363,D233,0081-CC04,0081,CC04,D233-CC04
364,D233,0081-CC05,0081,CC05,D233-CC05
365,D233,0081-CC06,0081,CC06,D233-CC06
498,D233,0081-CC07,0081,CC07,D233-CC07
66,D233,0346-CC03,0346,CC03,D233-CC08
320,D233,0346-CC09,0346,CC09,D233-CC09
